<a href="https://colab.research.google.com/github/dolDolSee/TF2.0/blob/main/TFPractice2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train.astype("float32"), x_test.astype("float32")
x_train,x_test = x_train.reshape([-1,784]), x_test.reshape([-1,784])
x_train, x_test = x_train/255., x_test/255.

11501568/11490434 [==============================] - 0s 0us/step


In [8]:
learning_rate = 0.02
training_epochs = 50
batch_size = 256
display_step = 1
examples_to_show = 10
input_size = 784
hidden1_size = 256
hidden2_size = 128

#비지도학습이기 때문에 y데이터는 묶어주지 않음.
train_data = tf.data.Dataset.from_tensor_slices(x_train)
train_data = train_data.shuffle(60000).batch(batch_size)

def random_normal_initializer_with_stddev_v1():
  return tf.keras.initializers.RandomNormal(mean=0.0, stddev=1.0, seed=None)

class AutoEncoder(tf.keras.Model):
  def __init__(self):
    super(AutoEncoder, self).__init__()
    self.hidden_layer_1 = tf.keras.layers.Dense(hidden1_size,
                                                activation="sigmoid",
                                                kernel_initializer = random_normal_initializer_with_stddev_v1(),
                                                bias_initializer = random_normal_initializer_with_stddev_v1())
    self.hidden_layer_2 = tf.keras.layers.Dense(hidden2_size,
                                                 activation="sigmoid",
                                                 kernel_initializer = random_normal_initializer_with_stddev_v1(),
                                                 bias_initializer=random_normal_initializer_with_stddev_v1())
    #decoding
    self.hidden_layer_3 = tf.keras.layers.Dense(hidden1_size,
                                                 activation="sigmoid",
                                                kernel_initializer = random_normal_initializer_with_stddev_v1(),
                                                bias_initializer = random_normal_initializer_with_stddev_v1()
                                                 )
    self.output_layer = tf.keras.layers.Dense(input_size,
                                                 activation="sigmoid",
                                                kernel_initializer = random_normal_initializer_with_stddev_v1(),
                                                bias_initializer = random_normal_initializer_with_stddev_v1())
  def call(self, x):
    H1_output = self.hidden_layer_1(x)
    H2_output = self.hidden_layer_2(H1_output)
    H3_output = self.hidden_layer_3(H2_output)
    reconstructed_x = self.output_layer(H3_output)
    return reconstructed_x

@tf.function
def mse_loss(y_pred, y_true):
  #autoencoder는 자기자신이 정답이기 때문에 input x값이 정답으로 쓰임.
  return tf.reduce_mean(tf.pow(y_true-y_pred,2))

optimizer = tf.optimizers.RMSprop(learning_rate)

@tf.function
def train_step(model, x):
  y_true = x
  with tf.GradientTape() as tape:
    y_pred = model(x)
    loss = mse_loss(y_pred, y_true)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

AutoEncodermodel = AutoEncoder()

for epoch in range(training_epochs):
  for batch_x in train_data:
    _, current_loss = train_step(AutoEncodermodel, batch_x), mse_loss(AutoEncodermodel(batch_x), batch_x)
  if epoch % display_step ==0:
    print("반복(Epoch): %d, 손실함수(Loss):%f" % ((epoch+1), current_loss))


반복(Epoch): 1, 손실함수(Loss):0.059783
반복(Epoch): 2, 손실함수(Loss):0.052657
반복(Epoch): 3, 손실함수(Loss):0.044145
반복(Epoch): 4, 손실함수(Loss):0.042305
반복(Epoch): 5, 손실함수(Loss):0.039264
반복(Epoch): 6, 손실함수(Loss):0.037796
반복(Epoch): 7, 손실함수(Loss):0.036886
반복(Epoch): 8, 손실함수(Loss):0.033664
반복(Epoch): 9, 손실함수(Loss):0.032793
반복(Epoch): 10, 손실함수(Loss):0.038032
반복(Epoch): 11, 손실함수(Loss):0.034777
반복(Epoch): 12, 손실함수(Loss):0.031568
반복(Epoch): 13, 손실함수(Loss):0.032581
반복(Epoch): 14, 손실함수(Loss):0.030912
반복(Epoch): 15, 손실함수(Loss):0.029501
반복(Epoch): 16, 손실함수(Loss):0.030939
반복(Epoch): 17, 손실함수(Loss):0.029298
반복(Epoch): 18, 손실함수(Loss):0.031637
반복(Epoch): 19, 손실함수(Loss):0.030132
반복(Epoch): 20, 손실함수(Loss):0.029538
반복(Epoch): 21, 손실함수(Loss):0.029178
반복(Epoch): 22, 손실함수(Loss):0.028434
반복(Epoch): 23, 손실함수(Loss):0.025688
반복(Epoch): 24, 손실함수(Loss):0.025839
반복(Epoch): 25, 손실함수(Loss):0.026385
반복(Epoch): 26, 손실함수(Loss):0.026806
반복(Epoch): 27, 손실함수(Loss):0.025039
반복(Epoch): 28, 손실함수(Loss):0.027120
반복(Epoch): 29, 손실함수(Loss):0.0